In [10]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from subprocess import check_output

#%matplotlib inline
datax = pd.read_csv('../Toru/train_quora_toy2.csv').fillna("")
#datax.head()

In [4]:
import re
import nltk
from nltk.corpus import stopwords
#from nltk.corpus import wordnet
#from nltk.stem import WordNetLemmatizer
STOP_WORDS = nltk.corpus.stopwords.words()

def clean_sentence(val):
    "remove chars that are not letters or numbers, downcase, then remove stop words"
    regex = re.compile('([^\s\w]|_)+')
    #sentence = regex.sub('', val).lower()
    sentence1 = regex.sub('', val)
    
    sentence1 = sentence1.split(" ")
    
    for word in list(sentence1):
        if word in STOP_WORDS:
            sentence1.remove(word)  
            
    sentence1 = " ".join(sentence1)
    
    # doing some substitutions after stopword removal and lower case
    sentence1 = re.sub(r'india', 'India', sentence1)
    sentence1 = re.sub(r'banglore', 'Bangalore', sentence1)
    sentence1 = re.sub(r'bangalore', 'Bangalore', sentence1)
    sentence1 = re.sub(r'pune', 'Pune', sentence1)
    sentence1 = re.sub(r'usa',  'United States', sentence1)
    sentence1 = re.sub(r'united states',  'United States', sentence1)
    sentence1 = re.sub(r'united states of america',  'United States', sentence1)
    #text = re.sub(r" USA ", " America ", text)
    #text = re.sub(r" u s ", " America ", text)
    #text = re.sub(r" uk ", " England ", text)
    #text = re.sub(r" UK ", " England ", text)
    #text = re.sub(r"india", "India", text)
    #text = re.sub(r"switzerland", "Switzerland", text)
    #text = re.sub(r"china", "China", text)
    
    return sentence1

def clean_dataframe(datax):
    "drop nans, then apply 'clean_sentence' function to question1 and 2"
    datax = datax.dropna(how="any")
    
    for col in ['question1', 'question2']:
        datax[col] = datax[col].apply(clean_sentence)
    
    return datax

datax = clean_dataframe(datax)
#print(datax)
##### Dropping a column created in later  cell ####
#data.drop('ques1', axis=1, inplace=True)
#data = data.drop('ques1', 1)

#places = GeoText('string')
#print(places.countries)

In [5]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

def build_corpus(datax):
    "Creates a list of lists containing words from each sentence"
    corpus = []
    for col in ['question1', 'question2']:
        for sentence1 in datax[col].iteritems():
            word_list = sentence1[1].split(" ")
            corpus.append(word_list)
            
    return corpus

corpus = build_corpus(datax)        
#print(corpus)

In [7]:
# changing logger level so that " under 10 batch_words" error does not pop up
import logging
logging.getLogger('gensim.models.word2vec').setLevel(logging.ERROR)

import collections
from collections import Counter
import math
# ignoring an warning
import warnings
#warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
#warnings.filterwarnings('ignore', '.*gensim.models.word2vec:under 10.*',)
import gensim

#print(len(corpus))


out_df = pd.DataFrame()
is_duplicate =[]
# Finding question pairs
for i in range(0, int((len(corpus))/2)):
    if i%5==0: 
        print(i)
    pair1 = list( corpus[i] for i in [i, i+ int((len(corpus))/2)] )
    pair1[0][0]=pair1[0][0].lower()
    pair1[1][0]=pair1[1][0].lower()
    #print(pair1)
    model = gensim.models.Word2Vec(pair1, min_count=1, workers=4)
    vec_avg1 = sum(model.wv[pair1[0]])/(len(pair1[0]))
    vec_avg2 = sum(model.wv[pair1[1]])/(len(pair1[1]))
    def cosine_similarity(vec_avg1,vec_avg2):
        "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
        sumxx, sumxy, sumyy = 0, 0, 0
        for j in range(len(vec_avg1)):
            x = vec_avg1[j]; y = vec_avg2[j]
            sumxx += x*x
            sumyy += y*y
            sumxy += x*y
        return sumxy/math.sqrt(sumxx*sumyy)
    #print(cosine_similarity(vec_avg1,vec_avg2))
    sim = cosine_similarity(vec_avg1,vec_avg2)
    if sim < 0: sim =0
        
    if pair1[0] ==[''] or pair1[1]==['']: # if any one question of the pair is empty then sim=0
        sim=0
    else:
        try:
            pos1 = nltk.pos_tag(pair1[0])
            #print(pos1)
        except IndexError:
            sim=sim
        try:
            pos2 = nltk.pos_tag(pair1[1])
            #print(pos2)
        except IndexError:
            sim=sim
        a = list()
        for item in pos1:
            if item[1] == 'NNP' or item[1]=='NN' or item[1]=='NNS' or item[1]=='NNPS': # considering any POS tag that starts with 'NN'
              a.append(item[0])
        if a != []:
            a = [item.lower() for item in a]
        else:
            sim=sim
        #print(a)
        b = list()
        for item in pos2:
            if item[1] == 'NNP' or item[1]=='NN' or item[1]=='NNS' or item[1]=='NNPS':
              b.append(item[0])
        if b != []:
            b = [item.lower() for item in b]
        else:
            sim=sim
        #print(b)

        if a==[] and b==[]:
            sim=0
        else:
            compare = lambda a, b: collections.Counter(a) == collections.Counter(b)
            #print(compare(a,b))
            if compare(a,b) == True:
                sim=1
            else:
                sim=0.17
    
    #print(sim)
    #out_df = pd.DataFrame({'is_duplicate': [sim]})
    is_duplicate.append(sim)
    i = i+1

#out_df.index.names = ['test_id'] 
out_df['test_id'] = datax['id']
out_df['is_duplicate'] = is_duplicate
out_df.to_csv('toy3_pred.csv', index=False)

0
5
10
15
[0.17, 0.17, 0.17, 0.17, 0, 0.17, 0, 1, 0.17, 0.17, 0.17, 1, 1, 1, 0.17, 0.17, 0.17, 0.17, 0.17, 0.17]


In [8]:
# this cell is used to find proper Noun-based similarity for the above cell
import collections
from collections import Counter

s1 = [['step', 'step', 'guide', 'invest', 'share', 'market', 'india'], ['step', 'step', 'guide', 'invest', 'share', 'market']]
s3 = [['story', 'kohinoor', 'kohinoor', 'diamond'], 
['would','happen','indian','government','stole','kohinoor','kohinoor','diamond','back']] 
s5 = [['make', 'physics', 'easy', 'learn'], ['make', 'physics', 'easy', 'learn']]
#s6 = ['make', 'physics', 'easy', 'learn']
s7 = [['first', 'sexual', 'experience', 'like'], ['first', 'sexual', 'experience']]
s9 = [['good', 'geologist'], ['great', 'geologist']]
s11= [['laws', 'change', 'status', 'student', 'visa', 'green', 'card', 'us', 'compare', 'immigration', 'laws', 'Canada'], 
      ['laws', 'change', 'status', 'student', 'visa', 'green', 'card', 'us', 'compare', 'immigration', 'laws', 'Japan']]
s13 = [['What', 'make', 'Physics', 'easy', 'learn'], ['How', 'make', 'physics', 'easy', 'learn']]

#model = Word2Vec(pairs1, size=100, window=20, min_count=200, workers=4)
model = gensim.models.Word2Vec(list(s9), min_count=1)

#print(model)
#model.wv['like']
#print(model.wv[s9[0]])

# parts-of-speech tag
pos1 = nltk.pos_tag(s11[0])
print(pos1)
pos2 = nltk.pos_tag(s11[1])
print(pos2)

count1= Counter([j for i,j in pos1])
print(count1)
count2= Counter([j for i,j in pos2])
print(count2)

#pos_difx = count1 - count2
#print(pos_difx)

count1.subtract(count2)
print(count1)

a = list()
for item in pos1:
    if item[1] == 'NNP': # considering any POS tag that starts with 'N'
      a.append(item[0])
print(a)
b = list()
for item in pos2:
    if item[1] == 'NNP':
      b.append(item[0])
print(b)

compare = lambda a, b: collections.Counter(a) == collections.Counter(b)
compare(a,b)

[('laws', 'NNS'), ('change', 'VBP'), ('status', 'NN'), ('student', 'NN'), ('visa', 'NN'), ('green', 'JJ'), ('card', 'NN'), ('us', 'PRP'), ('compare', 'VBP'), ('immigration', 'JJ'), ('laws', 'NNS'), ('Canada', 'NNP')]
[('laws', 'NNS'), ('change', 'VBP'), ('status', 'NN'), ('student', 'NN'), ('visa', 'NN'), ('green', 'JJ'), ('card', 'NN'), ('us', 'PRP'), ('compare', 'VBP'), ('immigration', 'JJ'), ('laws', 'NNS'), ('Japan', 'NNP')]
Counter({'NN': 4, 'NNS': 2, 'VBP': 2, 'JJ': 2, 'NNP': 1, 'PRP': 1})
Counter({'NN': 4, 'NNS': 2, 'VBP': 2, 'JJ': 2, 'NNP': 1, 'PRP': 1})
Counter({'NNP': 0, 'NN': 0, 'NNS': 0, 'PRP': 0, 'VBP': 0, 'JJ': 0})
['Canada']
['Japan']


False